#**100.01 Absa: Baseline(Coarse) Prediction**

1. Dataset: absa benchmark dataset (size: 2995)

     https://drive.google.com/file/d/12sYtvP1dvGCXnNOSkJKgJKtJgfbSgyXh/view?usp=sharing
2. Cadet: entity, attr detection 
3. Bert token classifier (0523) : evaltext span-marking, polarity prediciton
4. Prediction output: .json file, formatted as benchamrk set + an extra prediction key
5. Final predictions file is at 
```
指向情緒案/data/models/Bert_aspect_extraction/0610/benchmark_predictions.json
```
6. Update: 2021.6.11 by Nana
7. Issues: Bert token classifier is trained on some of the data in the benchmark set, so the evaltext span evaluation looks better 

## Preparing env & Cloning 

In [ ]:
# reload the modules by default 
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import torch
import random
import math
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import sys  
import pickle
from dataclasses import dataclass, field
from typing import Optional
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
from sklearn.utils import class_weight

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import sys
from matplotlib.font_manager import FontProperties
from matplotlib import font_manager

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
wkspace = '/content/gdrive/MyDrive/指向情緒案/data/models'

Mounted at /content/gdrive


## Loading Dataset

In [ ]:
dataset_file = '/content/gdrive/MyDrive/指向情緒案/data/annot_data/benchmark_set.json'
with open(dataset_file, 'r', encoding = 'utf-8') as fin:
    Datasets = json.load(fin)

In [ ]:
Datasets[0]

{'aspects': [{'attribute': '[通訊]頻段',
   'context_flag': True,
   'entity': '臺灣之星',
   'evaltext': '台星的態度就是在等宿主台哥，逸以待勞，準備寄生。',
   'rating': 1}],
 'thread': {'idx': 3260,
  'main': '5G第一階段價格標結束，4家電信-中華電信、台灣大、遠傳電信、亞太電信拚第3季開台，而台灣之星指出，尚未決定。中華電信總經理郭水義今日指出，將按照原先規劃在第三季開台。不過，中華電信董事長謝繼茂之前受訪指出，預定7月開台。而台灣大、遠傳也都會在第三季開台。亞太電信也表示，2800MHz將會在第三季開台。台灣之星指出，今日正式取得5G入場門票，下一階段將進入「頻位競標」，待兩階段5G競標程序皆完成後，將依規定啟動5G網路佈建，擴大網路容量、提升整體網路品質。至於何時開台？台灣之星指出，尚未決定。',
  'replies': ['台星的態度就是在等宿主台哥，逸以待勞，準備寄生。',
   '台星八成只能拿到沒有設備那部分的頻段比較晚開台也是滿合理的',
   '哎呀，你知道的太多了～～～反正暫時無資金可建設，還不如吃吃台哥大的豆腐比較實際！不然一開台，200億頻譜費得開始認列，但馬上就有10萬低資188用戶免費體驗，要加速賠到300億嗎？',
   '。台星免費體驗也沒說是那種體驗不期不待，沒有傷害。',
   '遠傳標80用那麼多錢，真的是拼面子。也令人感覺是在走一步算一步！日後開始5G價格戰，會很辛苦！',
   '亞太搞11賺設定費，每月還能收11元，比遠傳這個預付卡大戶省很多，這次遠傳標5g一點都沒徐旭東的樣子。',
   '可以是前2GB流量為5G不限速超量降回4G吃到飽每個月有2GB額度或是一個月給你2小時額度免費體驗5G,其他時間回復4G吃到飽它沒在合約說明如何的體驗,這些都叫體驗,因為你也有體驗到5G體驗完後變回4G畢竟跟台星簽的是4G合約,並不是簽5G合約',
   '每天1GB或6小時體驗(看哪個條件先達到)體驗時間每天00:00-06:00',
   '也不會沒資金吧~大大股東是首富「應該」不缺錢還有就是亞太跟台哥大談好了嗎要不然亞太的28GHz要拼命建基地台才有辦法開台吧?',
   '台星

## Cadet Prediction 
predictions at:` data/models/100.01cadet_coarse_pred.pkl`

In [ ]:
sys.path.append('casa/etc/jupyter') # casa/etc/jupyter/import_casa.py

In [ ]:
from import_casa import casa
font = FontProperties(fname=casa.get_data_path() / "font/NotoSansCJK-Regular.ttc")
plt.rcParams["font.family"] = "Microsoft YaHei" 

In [ ]:
import getpass, os
username = getpass.getpass("Github username: ")
password = getpass.getpass("Github password: ")
os.environ["CRED"] = f"{username}:{password}"
!git clone https://${CRED}@github.com/lopentu/casa
del username, password
!pip install -r /content/casa/requirements.txt

Github username: ··········
Github password: ··········
Cloning into 'casa'...
remote: Enumerating objects: 805, done.
remote: Counting objects: 100% (805/805), done.
remote: Compressing objects: 100% (507/507), done.
remote: Total 805 (delta 395), reused 643 (delta 243), pack-reused 0
Receiving objects: 100% (805/805), 4.72 MiB | 16.57 MiB/s, done.
Resolving deltas: 100% (395/395), done.
     |████████████████████████████████| 1.2MB 4.0MB/s 


In [ ]:
!pip install --upgrade gensim

In [ ]:
import gensim
from gensim.models import keyedvectors
dir(keyedvectors)

In [ ]:
from casa import Cadet
cadet = Cadet.load('/content/gdrive/MyDrive/指向情緒案/data/cadet/op20')
cadet

[INFO] 2021-06-10 04:01:02,831 gensim.utils: loading KeyedVectors object from /content/gdrive/MyDrive/指向情緒案/data/cadet/op20/ft-2020.kv
[INFO] 2021-06-10 04:01:04,946 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-06-10 04:01:04,948 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-06-10 04:01:04,950 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-06-10 04:01:04,953 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-06-10 04:01:05,112 gensim.utils: FastTextKeyedVectors lifecycle event {'fname': '/content/gdrive/MyDrive/指向情緒案/data/cadet/op20/ft-2020.kv', 'datetime': '2021-06-10T04:01:05.112305', 'gensim': '4.0.1', 'python': '3.7.10 (default, May  3 2021, 02:48:31) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


<Cadet: FastText(9851, 200), Seeds(103)>

In [ ]:
# data = cadet.detect("不過中華電信的吃到飽在山區收訊很差", verbose = False)

def get_entity(data):
  
    max_idx = data['entity_probs'].argmax()
    max_prob = data['entity_probs'][max_idx]
    max_ent = data['entity'][max_idx]
    return max_ent, max_prob

def get_service(data):
    max_idx = data['service_probs'].argmax()
    max_prob = data['service_probs'][max_idx]
    max_s = data['service'][max_idx]
    return max_s, max_prob


('資費方案', 0.4941969776370897)

In [ ]:
predictions = []
detection_errors = []
for i, data in enumerate(Datasets):
    th_idx =  data['thread']['idx']
    prediction = {'idx': th_idx , 'main':(), 'replies':[]} 
    th = data['thread']
    main = th['main']
    if main != '':
        main_data = cadet.detect(main)
        prediction['main'] = [get_entity(main_data), get_service(main_data)]
    replies = th['replies'] # a list of replies 
    for r in replies: 
        try:
          r_data = cadet.detect(r)
          prediction['replies'].append([get_entity(r_data), get_service(r_data)])
        except:
          # empty string, '在', 'NP直接去錄影帶店有439,覺得雙NP很懶要488但都不給', '有', ...
          detection_errors.append(i)
          prediction['replies'].append([]) # an empty list
    predictions.append(prediction)
    

In [ ]:
predictions[-1]

{'idx': 194,
 'main': [('其他電信', 0.4548851958463746), ('通訊品質', 0.9665581444005926)],
 'replies': [[('中華電信', 0.29193500060529604), ('其他', 0.9673461328737533)],
  [('其他電信', 0.5053107186577204), ('通訊品質', 0.4379179327266065)],
  [('台灣之星', 0.2666057966253697), ('通訊品質', 0.49395335928331824)],
  [('台灣之星', 0.2690545210462366), ('通訊品質', 0.470132899801794)],
  [('中華電信', 0.9995949218351506), ('資費方案', 0.967651210339059)],
  [('中華電信', 0.9432687167054168), ('資費方案', 0.9675599993793932)],
  [('其他電信', 0.19408513441546532), ('資費方案', 0.9643819728606264)]]}

In [ ]:
print(len(detection_errors))
print(detection_errors)
with open(f'{wkspace}/100.01cadet_coarse_pred.pkl', "wb") as fout:
    pickle.dump(predictions, fout)

46
[52, 176, 239, 278, 291, 322, 338, 463, 504, 504, 554, 554, 582, 614, 616, 639, 639, 694, 694, 705, 889, 906, 922, 943, 1011, 1021, 1052, 1072, 1075, 1122, 1123, 1210, 1210, 1243, 1265, 1265, 1265, 1302, 1320, 1323, 1323, 1351, 1395, 1413, 1417, 1437]


## Bert Prediction (aspect extraction)

### Loading model

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
model_path = f'{wkspace}/Bert_aspect_extraction/0523'
from transformers import BertTokenizerFast,AutoModel,BertForTokenClassification
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = BertForTokenClassification.from_pretrained(model_path, num_labels = 5) # O, B-VP, B-VN, I-VP, I-VN

In [ ]:
device = torch.device('cuda')
model.to(device)

### Preparing test dataloader

In [ ]:
# make a csv 
idx_list= []
prop_list= []
text_list= []
for data in Datasets:
    idx = data['thread']['idx']
    #### adding main ####
    idx_list.append(idx)
    prop_list.append(True)
    text_list.append(data['thread']['main'])
    ### adding replies ###
    replies_text = data['thread']['replies']
    for r in replies_text:
        idx_list.append(idx)
        prop_list.append(False)
        text_list.append(r)
assert len(idx_list) == len(prop_list) == len(text_list)

In [ ]:
import pandas as pd
data = {'idx':idx_list, 'is_main': prop_list, 'text': text_list}
df = pd.DataFrame(data, columns = ['idx','is_main', 'text'])
print(len(df)) # 13634
df.to_csv(f'{wkspace}/100.01bert_data.pkl')

13634


In [ ]:
import numpy as np
from datasets import ClassLabel, load_dataset, load_metric
texts = df['text'].tolist()
texts = [list(x) for x in texts]
# print(texts[0])
tokenized_examples = tokenizer(
    texts, 
    max_length=512,
    padding= "max_length",
    truncation=True,
    is_split_into_words=True, 
)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
tok_sents = torch.tensor(tokenized_examples['input_ids'])
tok_masks = torch.tensor(tokenized_examples['attention_mask'])
test_set = TensorDataset(tok_sents, tok_masks)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False, pin_memory=True)

### Evaluation and Prediction Pickling

In [ ]:
print("Evaluating Test Set ...")
model.eval()
test= {'preds':[], 'input_ids':[], 'logits':[]}
for batch in test_loader:
    with torch.no_grad():
        input_ids = batch[0].to(device) # input_ids
        attention_mask = batch[1].to(device)
        
        logits = model(input_ids, attention_mask=attention_mask).logits
        active_loss = attention_mask.view(-1) == 1
        logits = model(input_ids = input_ids, attention_mask=attention_mask).logits
        active_logits = logits.view(-1, 5) # num of labels=5
        
        _, pred = torch.max(logits, 2)
        test['preds'].extend(pred.cpu().numpy())
        test['input_ids'].extend(input_ids.cpu().numpy())
        test['logits'].extend(logits.cpu().numpy())

Evaluating Test Set ...



Bert Special tokens:
```
{'cls_token': '[CLS]', 101
 'mask_token': '[MASK]', 103
 'pad_token': '[PAD]', 0
 'sep_token': '[SEP]', 102
 'unk_token': '[UNK]' 100} 
```



In [ ]:
todaydir = '/content/gdrive/MyDrive/指向情緒案/data/models/Bert_aspect_extraction/0610'

In [ ]:
# Inference   
label_list = {0: 'B-VN', 1:'B-VP',  2:'I-VN', 3: 'I-VP', 4:'O'}
special_tokens = [101, 102] # 不應該拔UNK，只拔101, 102
predictions_all = test['preds']
inputids_all = test['input_ids']
### !!! 還是要手動切成一個字一個字，然後在tokenizer裡用is_split_into_words=False 
### !!! 把predictions的第一個token（CLS）跳掉，然後一直讀到倒數第二個（因為最後一個是SEP），然後把這個predictions和原始tokens zip起來
true_predictions = [[label_list[p] for (p, inpid) in zip(predsent, inpidsent) if inpid not in special_tokens] 
                    for predsent, inpidsent in zip(predictions_all, inputids_all)]
with open(os.path.join(todaydir, "100.01bert_preds.pkl"), 'wb') as f:
    pickle.dump(true_predictions, f)

## Final Predicitons 
combining Cadet and Bert 

In [ ]:
def translate(t):
    if t[-1] == 'P':
        return 'Positive'
    return 'Negative'

In [ ]:
# find the labeled span 
import more_itertools as mit
eval_spans = {}
thread_idxes = df['idx'].tolist()
assert len(texts) == len(true_predictions)
for idx, text, pred in zip(thread_idxes, texts, true_predictions):
      pred_spans_for_one_text = []
      pred = pred[:len(text)]
      pairs = [(i, val) for i, val in enumerate(pred) if val != 'O']
      pols = []
      if len(pairs) >1:
          polarity = list(map(lambda x: x[1], pairs))
          idxes = [x for (x, val) in pairs]
          idxes_groups = [list(group) for group in mit.consecutive_groups(idxes)]
          for group in idxes_groups:
              label = ''.join([text[i] for i in group])
              if len(label)>0: 
                  pred_spans_for_one_text.append(label)
                  pols.append(translate(pred[group[0]]))  # assume each group to have the same polarity
          print(text)
          print(pred_spans_for_one_text)
      # adding back to span dictionary 
      if idx not in eval_spans:
          eval_spans[idx] = []
          eval_spans[idx].append([pols, pred_spans_for_one_text]) 
      else: eval_spans[idx].append([pols, pred_spans_for_one_text]) 

In [ ]:
with open(f'{wkspace}/100.01cadet_coarse_pred.pkl', "rb") as fin:
    cadet_predictions = pickle.load(fin)



```
# cadet predictions structure 
{'idx': 194,
 'main': [('其他電信', 0.4548851958463746), ('通訊品質', 0.9665581444005926)],
 'replies': [[('中華電信', 0.29193500060529604), ('其他', 0.9673461328737533)],
  [('其他電信', 0.5053107186577204), ('通訊品質', 0.4379179327266065)],
  [('台灣之星', 0.2666057966253697), ('通訊品質', 0.49395335928331824)],
  [('台灣之星', 0.2690545210462366), ('通訊品質', 0.470132899801794)],
  [('中華電信', 0.9995949218351506), ('資費方案', 0.967651210339059)],
  [('中華電信', 0.9432687167054168), ('資費方案', 0.9675599993793932)],
  [('其他電信', 0.19408513441546532), ('資費方案', 0.9643819728606264)]]}

# benchmark structure 
"aspects": [
      {
        "entity": "臺灣之星",
        "attribute": "[通訊]頻段",
        "evaltext": "台星的態度就是在等宿主台哥，逸以待勞，準備寄生。",
        "rating": 1,
        "context_flag": true
      }
    ]
```



In [ ]:
final_predictions = {}
for pred in cadet_predictions:
   # every pred is one thread 
    idx = pred['idx']
    # corresponding span data
    asptuples = []
    # if main data's span is not empty
    if len(eval_spans[idx][0][1]) >1:
        pred_ent = pred['main'][0][0]
        pred_serv = pred['main'][1][0]
        ################################
        pred_ent_prob = round(pred['main'][0][1], 4)
        pred_serv_prob = round(pred['main'][1][1], 4)
        asptuples.append({'entity':pred_ent, 'attribute': pred_serv, 'evaltext':eval_spans[idx][0][1] ,'polarity':eval_spans[idx][0][0]})
    rep_spans = eval_spans[idx][1:]
    for j, rep_spans in enumerate(rep_spans):
        # if text spans ar edetected
        if len(rep_spans[1]) > 1:
            ent = pred['replies'][j][0][0]
            serv = pred['replies'][j][1][0]
            #################################
            ent_prob = round(pred['replies'][j][0][1], 4)
            serv_prob = round(pred['replies'][j][1][1], 4)
            asptuples.append({'entity':ent, 'attribute':serv, 'evaltext':rep_spans[1] ,'polarity': rep_spans[0]})
    # one thread index should only appear once
    final_predictions[idx] = asptuples 

In [ ]:
with open(f'{todaydir}/100.01_final_predictions.pkl', "wb") as fout:
    pickle.dump(final_predictions, fout)

## The final predictions file is at 
```
指向情緒案/data/models/Bert_aspect_extraction/0610/benchmark_predictions.json
```



In [ ]:
# final_predictions[2232]
for i in range(len(Datasets)):
    idx = Datasets[i]['thread']['idx']
    Datasets[i]['prediction'] = final_predictions[idx]
# workspace = '/content/gdrive/MyDrive/指向情緒案/data'
with open(f'{todaydir}/benchmark_predictions.json', 'w', encoding = 'utf-8') as fout:
    json.dump(Datasets, fout, ensure_ascii=False, indent=2)